# Automated Project: Planning, Estimation, and Allocation

In [3]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
from dotenv import load_dotenv

import os
import yaml
from crewai import Agent, Task, Crew

In [4]:
load_dotenv()
openai_api_key=os.getenv('OPENAI_API_KEY')

### Set OpenAI Model

In [5]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

### Loading Tasks and Agents YAML files

In [6]:
# Define file paths for yaml configuration
files = {
    'agents':'config/agents.yaml',
    'tasks':'config/tasks.yaml'
}

In [7]:
# Load configurations from yaml files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

In [8]:
configs

{'agents': {'project_planning_agent': {'role': 'The Ultimate Project Planner\n',
   'goal': 'To meticulously break down the {project_type} project into actionable tasks, ensuring no detail is overlooked, and setting precise timelines that align with the {project_objectives}.\n',
   'backstory': "As a veteran project manager, you’ve led numerous successful projects, particularly in {industry}. Your keen eye for detail and strategic thinking have always ensured that projects are delivered on time and within scope. Now, you're tasked with planning the next groundbreaking {project_type} project.\n",
   'allow_delegation': False,
   'verbose': True},
  'estimation_agent': {'role': 'Expert Estimation Analyst\n',
   'goal': 'Provide highly accurate time, resource, and effort estimations for each task in the {project_type} project to ensure it is delivered efficiently and on budget.\n',
   'backstory': 'You are the go-to expert for project estimation in {industry}. With a wealth of experience 

In [9]:
# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

### Create Pydantic Models for Structured Output

In [10]:
from typing import List
from pydantic import BaseModel, Field

In [11]:
class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")

### Create Crew, Agents and Tasks

In [12]:
# Creating Agents
project_planning_agent = Agent(
  config=agents_config['project_planning_agent']
)

estimation_agent = Agent(
  config=agents_config['estimation_agent']
)

resource_allocation_agent = Agent(
  config=agents_config['resource_allocation_agent']
)

In [13]:
# Creating Tasks
task_breakdown = Task(
  config=tasks_config['task_breakdown'],
  agent=project_planning_agent
)

time_resource_estimation = Task(
  config=tasks_config['time_resource_estimation'],
  agent=estimation_agent
)

resource_allocation = Task(
  config=tasks_config['resource_allocation'],
  agent=resource_allocation_agent,
  output_pydantic=ProjectPlan # This is the structured output we want
)

In [14]:
# Creating Crew
crew = Crew(
  agents=[
    project_planning_agent,
    estimation_agent,
    resource_allocation_agent
  ],
  tasks=[
    task_breakdown,
    time_resource_estimation,
    resource_allocation
  ],
  verbose=True
)

### Crew's Inputs

In [15]:
from IPython.display import display, Markdown

In [16]:
project = 'Website'
industry = 'Technology'
project_objectives = 'Create a website for a small business'
team_members = """
- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)
"""
project_requirements = """
- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust
"""

In [17]:
# Format the dictionary as Markdown for a better display in Jupyter Lab
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}
**Project Requirements:**
{project_requirements}
"""

In [18]:
# Display the formatted output as Markdown
display(Markdown(formatted_output))


**Project Type:** Website

**Project Objectives:** Create a website for a small business

**Industry:** Technology

**Team Members:**

- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)

**Project Requirements:**

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust



### Kicking off the crew

In [19]:
# The given Python dictionary
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'team_members': team_members,
  'project_requirements': project_requirements
}

In [21]:
# Run the crew
result = crew.kickoff(
  inputs=inputs
)

# Agent: The Ultimate Project Planner
## Task: Carefully analyze the project_requirements for the Website project and break them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are accounted for:

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build tru

### Usage Metrics and Costs
Let’s see how much it would cost each time if this crew runs at scale.

In [22]:
import pandas as pd

In [23]:
costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0022


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,14451,7007,0,7444,5


### Result

In [24]:
result.pydantic.dict()

{'tasks': [{'task_name': 'Define Project Objectives and Scope',
   'estimated_time_hours': 40.0,
   'required_resources': ['John Doe']},
  {'task_name': 'Create a Responsive Design',
   'estimated_time_hours': 80.0,
   'required_resources': ['Bob Smith', 'Jane Doe']},
  {'task_name': 'Design the User Interface',
   'estimated_time_hours': 80.0,
   'required_resources': ['Bob Smith']},
  {'task_name': 'Develop a Navigation System',
   'estimated_time_hours': 40.0,
   'required_resources': ['Jane Doe']},
  {'task_name': "Create an 'About Us' Page",
   'estimated_time_hours': 40.0,
   'required_resources': ['Jane Doe']},
  {'task_name': "Design a 'Services' Page",
   'estimated_time_hours': 40.0,
   'required_resources': ['Jane Doe']},
  {'task_name': "Create a 'Contact Us' Page",
   'estimated_time_hours': 40.0,
   'required_resources': ['Jane Doe']},
  {'task_name': 'Develop a Blog Section',
   'estimated_time_hours': 80.0,
   'required_resources': ['Jane Doe']},
  {'task_name': 'Optimi

### Inspect further

In [25]:
tasks = result.pydantic.dict()['tasks']
df_tasks = pd.DataFrame(tasks)

# Display the DataFrame as an HTML table
df_tasks.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,task_name,estimated_time_hours,required_resources
0,Define Project Objectives and Scope,40.000000,['John Doe']
1,Create a Responsive Design,80.000000,"['Bob Smith', 'Jane Doe']"
2,Design the User Interface,80.000000,['Bob Smith']
3,Develop a Navigation System,40.000000,['Jane Doe']
4,Create an 'About Us' Page,40.000000,['Jane Doe']
5,Design a 'Services' Page,40.000000,['Jane Doe']
6,Create a 'Contact Us' Page,40.000000,['Jane Doe']
7,Develop a Blog Section,80.000000,['Jane Doe']
8,Optimize for SEO and Loading Times,40.000000,['Jane Doe']
9,Integrate Social Media Links,40.000000,['Jane Doe']


### Inspecting Milestones

In [26]:
milestones = result.pydantic.dict()['milestones']
df_milestones = pd.DataFrame(milestones)

# Display the DataFrame as an HTML table
df_milestones.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,milestone_name,tasks
0,Project Initiation,['Define Project Objectives and Scope']
1,Design Phase Completion,"['Create a Responsive Design', 'Design the User Interface']"
2,Development Phase Completion,"['Develop a Navigation System', ""Create an 'About Us' Page"", ""Design a 'Services' Page"", ""Create a 'Contact Us' Page"", 'Develop a Blog Section']"
3,Optimization Phase Completion,"['Optimize for SEO and Loading Times', 'Integrate Social Media Links', 'Create a Testimonials Section']"
4,Testing Phase Completion,['Quality Assurance Testing']
5,Launch Phase Completion,['Website Launch and Monitoring']
